In [7]:
import numpy as np

# -----------------
# 1. INITIAL SETUP
# -----------------

# Initial species counts (internal variables that change)
# Includes internal species and the interface species S8
# Order: [N4, N5, N8, N10, N11] -> N_total = 5 changing species
species = np.array([100, 50, 20, 150, 75])
N_total = len(species)

# Stoichiometry Matrix (M x R: M species, R reactions)
# M = 5 changing species. R = 9 reactions (R1..R3, R6..R7, R9, R_int, plus reverses)
# Rows: N4, N5, N8, N10, N11
# Columns: R1f, R1r, R2f, R2r, R3, R6f, R6r, R7f, R7r, R9f, R8f, R8r
# Note: R1, R3, R6, R7 are exchanges, their external species are assumed constant.
stoichiometry = np.array([
    [1, -1, -1, 1, 0, 0, 0, 0, 0, 0, -1, 1],  # Change in N4
    [0, 0, 1, -1, -1, 0, 0, 0, 0, 0, 1, -1],  # Change in N5
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1],  # Change in N8 (Interface)
    [0, 0, 0, 0, 0, 1, -1, -1, 1, -1, 0, 0],  # Change in N10
    [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0]   # Change in N11
])

# External Chemostat Concentrations (Assumed constant for propensities)
C_ext = {'S1': 1000, 'S2': 500, 'S3': 1000, 'S6': 1000, 'S7': 1000, 'S9': 1000}

# Arbitrary Rate Constants (allowing non-linear terms)
k = np.array([1.0, 0.1, 0.001, 0.005, 0.5, 1.0, 0.1, 1.0, 0.1, 0.01, 0.05, 0.1])

# -----------------
# 2. PROPENSITY FUNCTION (The Non-linear/Mass-Action Core)
# -----------------
def get_propensities(N, k, C_ext):
    N4, N5, N8, N10, N11 = N
    C1, C2, C3, C6, C7, C9 = C_ext['S1'], C_ext['S2'], C_ext['S3'], C_ext['S6'], C_ext['S7'], C_ext['S9']

    # Array of reaction propensities h_rho
    h = np.zeros(len(k))

    # Module 1 (Non-linear R2 terms)
    h[0] = k[0] * C1  # R1f: S1 -> X4
    h[1] = k[1] * N4   # R1r: X4 -> S1
    h[2] = k[2] * N4 * N5 # R2f: X4 + X5 -> 2X5  (Non-linear)
    h[3] = k[3] * N5 * N5 # R2r: 2X5 -> X4 + X5 (Non-linear)
    h[4] = k[4] * N5 # R3: X5 -> S3

    # Module 2
    h[5] = k[5] * C6 # R6f: S6 -> X10
    h[6] = k[6] * N10 # R6r: X10 -> S6
    h[7] = k[7] * N10 # R7f: X10 -> S7
    h[8] = k[8] * C7 # R7r: S7 -> X10
    h[9] = k[9] * N10 * N11 # R9f: X10 + X11 -> ... (Non-linear)

    # Interface Reaction (Module 1 <-> Module 2)
    h[10] = k[10] * N8 * N4 # R8f: S8 + X4 -> X5 (Non-linear link)
    h[11] = k[11] * N5 * N5 # R8r: X5 -> S8 + X4 (Non-linear link)
    
    # Must ensure propensities are non-negative
    return np.maximum(h, 0.0)

# -----------------
# 3. GILLESPIE SIMULATION LOOP
# -----------------
def run_gillespie(initial_species, k, C_ext, max_time):
    time = 0.0
    N = initial_species.copy()
    
    # Simple logging structure
    time_points = [0.0]
    species_counts = [N.copy()]

    while time < max_time:
        h = get_propensities(N, k, C_ext)
        a0 = np.sum(h)

        if a0 == 0:
            break  # No more reactions can occur

        # 1. Time step (Exponential distribution)
        tau = np.random.exponential(1.0 / a0)
        time += tau

        # 2. Reaction selection
        # Choose reaction rho with probability h_rho / a0
        rho = np.random.choice(len(h), p=(h / a0))

        # 3. Update species counts
        N = N + stoichiometry[:, rho]
        
        # Log the state
        time_points.append(time)
        species_counts.append(N.copy())

    return np.array(time_points), np.array(species_counts)

# Example Run (Time-consuming step)
# time_points, species_counts = run_gillespie(species, k, C_ext, max_time=100.0)

In [10]:
import numpy as np

# --- 1. INPUTS: PRE-CALCULATED MATRICES ---

# R^(1) and R^(2) are the fundamental resistance matrices (3x3) calculated 
# from the steady states of the isolated modules. (Placeholder values used)
R_1 = np.array([
    [10.0, 1.0, 0.0], 
    [1.0, 15.0, 2.0], 
    [0.0, 2.0, 20.0]
]) 
R_2 = np.array([
    [30.0, 3.0, 0.0], 
    [3.0, 35.0, 4.0], 
    [0.0, 4.0, 40.0]
]) 

# Pi^trans (Π^T): The transformation matrix (6x3) that maps R^(1) from its 
# fundamental space to the full external current space (N_curr=6).
# (Placeholder values for a typical serial connection)
Pi_trans_1_to_3 = np.array([
    [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], # Maps to first 3 currents
    [0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]  # Maps to next 3 currents (interface)
])

# S^(3): The Selection Matrix (6x3) that projects the total 6 currents (I_ext) 
# onto the 3 fundamental currents (I_fund).
# (Placeholder values for a typical serial connection)
S_3 = np.array([
    [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], 
    [0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]
]) 

# --- 2. CIRCUIT CALCULATION ---

# Step A: Transform R_1 into the full redundant 6x6 current space (R_1')
# R_1_prime = Π^trans * R_1 * (Π^trans)^T
R_1_prime = np.dot(Pi_trans_1_to_3, np.dot(R_1, Pi_trans_1_to_3.T)) 

# Step B: Expand R_2 into the full redundant 6x6 current space (R_2_expanded)
# R_2 is placed in the specific 3x3 block it affects (e.g., lower right corner for a serial connect).
R_2_expanded = np.zeros((6, 6))
R_2_expanded[3:, 3:] = R_2 

# Step C: Combine R_1' and R_2_expanded (R_intermediate)
R_intermediate = R_1_prime + R_2_expanded

# Step D: Project the result down to the 3x3 fundamental space (R_Circuit)
# R_Circuit = S^3^T * R_intermediate * S^3
R_Circuit = np.dot(S_3.T, np.dot(R_intermediate, S_3))

# --- 3. OUTPUT ---

print("Final R^Circuit (3x3 Fundamental Resistance Matrix):")
print(R_Circuit.round(4))

Stationary State (N4, N5, N8, N10, N11):
[5250.3473  949.6487  344.1581 1000.        0.    ]

Jacobian Matrix J (5x5):
[[ -18.2576  194.1759 -262.5174    0.        0.    ]
 [  18.1576 -194.6759  262.5174    0.        0.    ]
 [ -17.2079  189.9297 -262.5174    0.        0.    ]
 [   0.        0.        0.       -1.1     -10.    ]
 [   0.        0.        0.        0.      -10.    ]]


--- CONSISTENCY CHECK ABORTED ---
The crucial step of projecting the internal 5x5 Resistance Matrix (derived from J) onto the 3x3 Fundamental External Resistance Matrix requires the full Cycle Matrix (C) and external stoichiometry (nabla_Y), which cannot be calculated from the minimal retained attributes. 

To verify the theory, the calculated R_Direct must be structurally related to the R_Circuit through the paper's full set of transformation matrices (C, S, pi, l).
